In [1]:
! pip install beautifulsoup4
! pip install python-slugify

In [2]:
import json
import urllib.request
from bs4 import BeautifulSoup

from pprint import pprint

from slugify import slugify


In [3]:
url = "https://www.fragrantica.com/notes/"

headers={
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.4 Safari/605.1.15',
}

In [4]:
# https://stackoverflow.com/questions/60878105/web-scraping-using-python-urlopen-returns-http-error-403-forbidden
request = urllib.request.Request(url, None, headers)
response = urllib.request.urlopen(request)

In [5]:
html = response.read()

In [6]:
soup = BeautifulSoup(html, 'html.parser')

In [7]:
content = soup.select('div.grid-x.grid-padding-y.grid-margin-y.grid-padding-x.grid-margin-x')[0]

In [8]:
l = content.findChildren("div", recursive=False)

In [9]:
notes = [l[i:i + 3] for i in range(0, len(l), 3)]

In [10]:
for i in notes:
  category = i[0].text.strip().title()
  print(category)

Notes
Citrus Smells
Fruits, Vegetables And Nuts
Flowers
White Flowers
Greens, Herbs And Fougeres
Spices
Sweets And Gourmand Smells
Woods And Mosses
Resins And Balsams
Musk, Amber, Animalic Smells
Beverages
Natural And Synthetic, Popular And Weird
Uncategorized


In [11]:
category_order = {j:i for i, j in enumerate([i[0].text.strip().title().replace(" Smells", "") for i in notes])}

category_order

{'Notes': 0,
 'Citrus': 1,
 'Fruits, Vegetables And Nuts': 2,
 'Flowers': 3,
 'White Flowers': 4,
 'Greens, Herbs And Fougeres': 5,
 'Spices': 6,
 'Sweets And Gourmand': 7,
 'Woods And Mosses': 8,
 'Resins And Balsams': 9,
 'Musk, Amber, Animalic': 10,
 'Beverages': 11,
 'Natural And Synthetic, Popular And Weird': 12,
 'Uncategorized': 13}

In [12]:
data = dict()

for i in notes:
  category = i[0].text.strip().title().replace(" Smells", "")
  print(category)

  for notebox in i[2].find_all("div", {'class': 'notebox'}):
    note = notebox.find("a")

    if note.contents[-1]:

      note_name = note.text.strip()
      if note_name.lower() == note_name:  note_name = note_name.title()
      if note_name in data: note_name += " "

      data[note_name] = dict()

      data[note_name]["name"] = note_name

      if note_name == "Fuji Apple":
        data[note_name]["category"] = "Fruits, Vegetables And Nuts"
      elif note_name == "Soy Milk":
        data[note_name]["category"] = "Beverages"
      elif note_name == "Sudachi citrus":
        data[note_name]["category"] = "Citrus"
      else:
        data[note_name]["category"] = category

      data[note_name]["url"] = note["href"]
      data[note_name]["img"] = note.contents[0]["src"].split("?")[0]
      print(note_name)
  print("\n")

Notes


Citrus
Bergamot
Bigarade
Bitter Orange
Blood Orange
Calamansi
Candied Lemon
Chen Pi
Chinotto
Citron
Citrus Water
Citruses
Clementine
Finger Lime
Grapefruit
Grapefruit Leaf
Grapefruit Peel
Green Tangerine
Hassaku
Hatkora Lemon
Kaffir Lime
Kumquat
Lemon
Lemon Balm
Lemon Tree
Lemon Verbena
Lemon Zest
Lemongrass
Lime
Limetta
Litsea Cubeba
Mandarin Orange
Mandora
Methyl Pamplemousse
Neroli
Orange
Palestinian Sweet Lime
Petitgrain
Pokan
Pomelo
Rangpur
Red Mandarin
Shiikuwasha
Tangelo
Tangerine
Tangerine Zest
Yuzu


Fruits, Vegetables And Nuts
Acai Berry
Acerola
Acorn
Almond
Apple
Apple Juice
Apple Pulp
Apple Sherbet
Apricot
Arctic Bramble
Argan
Artichoke
Banana
Barberry
Bearberry
Beetroot
Black Cherry
Black Currant
Black Sapote
Black Walnut
Blackberry
Blackthorn
Blueberry
Boysenberry
Brazil Nut
Breadnut
Buriti
Burning Cherry
Candlenut
Cantaloupe
Carambola (Star Fruit)
Carrot
Cashew
Cassowary Fruit
Cauliflower
Cep
Cepes
Chayote
Cherimoya
Cherry
Chestnut
Chia Seed
Chickpeas
Chinese Mag

In [13]:
[(c, sum(1 for i in data.values() if i["category"] == c)) for c in {i["category"] for i in data.values()}]

[('Woods And Mosses', 134),
 ('Flowers', 311),
 ('Natural And Synthetic, Popular And Weird', 231),
 ('Citrus', 47),
 ('Resins And Balsams', 40),
 ('Greens, Herbs And Fougeres', 203),
 ('Spices', 66),
 ('White Flowers', 27),
 ('Sweets And Gourmand', 126),
 ('Fruits, Vegetables And Nuts', 189),
 ('Beverages', 86),
 ('Musk, Amber, Animalic', 52)]

In [14]:
print(f"TOTAL {len(data)}")

TOTAL 1512


In [15]:
sdata = dict(sorted(data.items(), key=lambda x: (category_order[x[1]["category"]], slugify(x[0].lower()))))

In [16]:
with open('notes.json', 'w') as f:
    s = json.dumps(sdata, indent=4)
    f.write(s)